In [1]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")
DATABASE = os.getenv("DATABASE")
URL = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

ENGINE = create_engine(URL)

# Fetching constraints of database

In [2]:
query = """
SELECT 
    conname AS constraint_name,
    contype AS constraint_type,
    conrelid::regclass AS table_name,
    CASE 
        WHEN contype IN ('f', 'c') THEN confrelid::regclass::text
        ELSE NULL
    END AS referenced_table,
    conkey AS constraint_columns,
    confkey AS referenced_columns
FROM 
    pg_constraint
WHERE 
    conrelid::regclass::text NOT LIKE 'pg_%'
ORDER BY 
    conrelid::regclass::text, conname
"""

with ENGINE.connect() as conn:
    constraints = pd.DataFrame(conn.execute(text(query)))

constraints

,constraint_name,constraint_type,table_name,referenced_table,constraint_columns,referenced_columns
0,cardinal_number_domain_check,c,-,-,None,None
1,year_check,c,-,-,None,None
2,yes_or_no_check,c,-,-,None,None
3,actor_pkey,p,actor,None,[1],None
4,address_pkey,p,address,None,[1],None
5,fk_address_city,f,address,city,[5],[1]
6,category_pkey,p,category,None,[1],None
7,city_pkey,p,city,None,[1],None
8,fk_city,f,city,country,[3],[1]
9,country_pkey,p,country,None,[1],None


# Fetching column list from each table

In [3]:
query = """
SELECT
    c.table_schema,
    c.table_name,
    string_agg(c.column_name || ' (' || c.data_type || ')', ', ' ORDER BY c.ordinal_position) AS column_list
FROM
    information_schema.columns AS c
JOIN
    information_schema.tables AS t
    ON c.table_name = t.table_name
    AND c.table_schema = t.table_schema
WHERE
    t.table_type = 'BASE TABLE'
    AND t.table_schema NOT IN ('information_schema', 'pg_catalog')
GROUP BY
    c.table_schema,
    c.table_name
ORDER BY
    c.table_schema,
    c.table_name;
"""

with ENGINE.connect() as conn:
    columns = pd.DataFrame(conn.execute(text(query)))

columns[['table_name', 'column_list']].to_csv("tables.txt", index=False, sep="\t", header=False)
columns

,table_schema,table_name,column_list
0,public,actor,"actor_id (integer), first_name (character vary..."
1,public,address,"address_id (integer), address (character varyi..."
2,public,category,"category_id (integer), name (character varying..."
3,public,city,"city_id (integer), city (character varying), c..."
4,public,country,"country_id (integer), country (character varyi..."
5,public,customer,"customer_id (integer), store_id (smallint), fi..."
6,public,film,"film_id (integer), title (character varying), ..."
7,public,film_actor,"actor_id (smallint), film_id (smallint), last_..."
8,public,film_category,"film_id (smallint), category_id (smallint), la..."
9,public,inventory,"inventory_id (integer), film_id (smallint), st..."


# Saving constraint of database

In [4]:
for col in columns['table_name']:
    consts = constraints[constraints['table_name'] == col]
    for c in consts.iterrows():
        if c[1]['constraint_type'] == 'p':
            with open('../constraints.txt', 'a') as f:

                table_name = c[1]['table_name']
                column_list = columns[columns['table_name'] == table_name]['column_list'].values[0].split(', ')
                if len(c[1]['constraint_columns']) == 1:
                    constrained_index = int(c[1]['constraint_columns'][0]-1)
                else:
                    continue
                column_name = column_list[constrained_index]

                f.write(f"Column {column_name} is a primary key of the table {table_name}\n")
        elif c[1]['constraint_type'] == 'f':
            with open('../constraints.txt', 'a') as f:
                table_name = c[1]['table_name']
                column_list = columns[columns['table_name'] == table_name]['column_list'].values[0].split(', ')
                if len(c[1]['constraint_columns']) == 1:
                    constrained_index = int(c[1]['constraint_columns'][0]-1)
                else:
                    continue
                column_name = column_list[constrained_index]

                referenced_table = c[1]['referenced_table']
                referenced_columns = c[1]['referenced_columns']
                referenced_column_list = columns[columns['table_name'] == referenced_table]['column_list'].values[0].split(', ')
                referenced_column_name = referenced_column_list[referenced_columns[0]-1]

                f.write(f"Column {column_name} is a foreign key of the table {table_name} and references column {referenced_column_name} of the table {referenced_table}\n")


In [6]:
from dotenv import load_dotenv
import pandas as pd
import os
from sqlalchemy import create_engine

load_dotenv()

username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
database = os.getenv('DATABASE')
host = os.getenv('HOST')
port = os.getenv('PORT')
url = f'postgresql://{username}:{password}@{host}:{port}/{database}'

engine = create_engine(url)

query = 'SELECT category, COUNT(category) AS count FROM film_category JOIN film ON film_category.film_id = film.film_id GROUP BY category'
data = pd.read_sql(query, engine)

import matplotlib.pyplot as plt
from matplotlib import pyplot

plt.pie(data['count'], labels=data['category'])
plt.axis('equal')
plt.title('Pie Chart of Number of Films in Each Category')
plt.show()
print("Code Executed Successfully!")

ProgrammingError: (psycopg2.errors.UndefinedColumn) BŁĄD:  kolumna "category" nie istnieje
LINE 1: SELECT category, COUNT(category) AS count FROM film_category...
               ^
HINT:  Być może chodziło ci o wskazanie kolumny "film_category.category_id".

[SQL: SELECT category, COUNT(category) AS count FROM film_category JOIN film ON film_category.film_id = film.film_id GROUP BY category]
(Background on this error at: https://sqlalche.me/e/20/f405)